In [ ]:
# !wget https://raw.githubusercontent.com/alexmascension/revisit_reynolds_fb/master/requirements.txt
# !pip install -r requirements.txt

In [ ]:
import scanpy as sc
import scanpy.external as sce

import pandas as pd
import numpy as np
import os
from functools import reduce
import gseapy as gp

import triku as tk

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import scipy.stats as sts

In [ ]:
# To print versions of imports 

import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

excludes = ['builtins', 'types', 'sys']

imported_modules = [module for module in imports() if module not in excludes]

clean_modules = []

for module in imported_modules:

    sep = '.'  # to handle 'matplotlib.pyplot' cases
    rest = module.split(sep, 1)[0]
    clean_modules.append(rest)

changed_imported_modules = list(set(clean_modules))  # drop duplicates

pip_modules = !pip freeze  # you could also use `!conda list` with anaconda

for module in pip_modules:
    try:
        name, version = module.split('==')
        if name in changed_imported_modules:
            print(name + '\t' + version)
    except:
        pass

In [ ]:
seed = 0

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 150

In [ ]:
def assign_cats(adata, dict_cats, column_groupby='leiden', quantile_gene_sel=0.7, do_return=False, intermediate_states=False, diff=0.05, 
                key_added='assigned_cats', min_score=50):
    """
    This functions uses a set of genes assigned to different categories so that leiden clusters can be assigned to one of these categories.
    For example, to categorize fibroblasts from pericytes, endothelial cells, or cells with high mitochondrial content.
    It could be done with each cell individually, but it is better to use clusters to discern the different categories because
    the method, although efficient, can sometimes be noisy due to the noisiness of the sc datasets.
    """
    
    for cat in list(dict_cats.keys()):
        mat_cat = np.zeros((len(adata), len(dict_cats[cat])), dtype=float)
        
        for gene_idx, gene in enumerate(dict_cats[cat]):
            try:
                mat_cat[:, gene_idx] = np.asarray(np.dot(adata.obsp['connectivities'], adata[:, gene].X).todense()).ravel() / adata.uns['neighbors']['params']['n_neighbors']
                
                # We normalize the expression to the maximum (per 98) of expression ** 0.5 to account to some extent for the expression of the gene. 
                # In the end, a gene highly expressed must account for more than a gene with lower expression
                mat_cat[mat_cat[:, gene_idx] > 0, gene_idx] = np.argsort(np.argsort(mat_cat[mat_cat[:, gene_idx] > 0, gene_idx]))
                mat_cat[:, gene_idx] /= np.max(mat_cat[:, gene_idx])
            except:
                print(f'Gene {gene} is not on the list')    
            
        sum_mat_cat = np.asarray(mat_cat.mean(1)).ravel()       
        adata.obs[cat] = sum_mat_cat
    
    score_per_cluster = adata.obs[[column_groupby] + list(dict_cats.keys())].groupby(column_groupby).quantile(quantile_gene_sel)
    max_cat_dict_std = dict(zip(score_per_cluster.std(1).index, score_per_cluster.std(1).values))
    adata.obs[f'{key_added}_std'] = [max_cat_dict_std[i] for i in adata.obs[column_groupby]]
    max_cat_dict_mean = dict(zip(score_per_cluster.mean(1).index, score_per_cluster.mean(1).values))
    adata.obs[f'{key_added}_mean'] = [max_cat_dict_mean[i] for i in adata.obs[column_groupby]]
    adata.obs[f'{key_added}_CV'] = adata.obs[f'{key_added}_mean'] / adata.obs[f'{key_added}_std']
    
    for cat in score_per_cluster.columns:
        max_cat_dict = dict(zip(score_per_cluster.index, score_per_cluster[cat].values))
        adata.obs[f'{key_added}_{cat}'] = [max_cat_dict[i] for i in adata.obs[column_groupby]]
    
    if intermediate_states: # For each cluster we will identify which categories are close to the highest one, and merge their names.
        list_names_cats_per_cluster = []
        for cluster in score_per_cluster.index:
            scores_cluster = score_per_cluster.loc[cluster]
            scores_cluster = scores_cluster[scores_cluster > scores_cluster.max() - diff]
            list_names_cats_per_cluster.append('/'.join(scores_cluster.index))
            
        inter_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, list_names_cats_per_cluster))
        adata.obs[f'{key_added}'] = [str(inter_cat_dict[i]) for i in adata.obs[column_groupby]]
    else:
        max_cat_dict = dict(zip(score_per_cluster.idxmax(axis=1).index, score_per_cluster.idxmax(axis=1).values))
        adata.obs[f'{key_added}'] = [str(max_cat_dict[i]) for i in adata.obs[column_groupby]]
    
    if do_return:
        return score_per_cluster

# Reynolds et al. 2020 dataset download and preprocess
In this section we are going to download and process the anndata files for the 5 healthy samples (S1 to S5). We are first going to use the processed files from Reynolds et al., because they contain the main fb populations, which we are interested in (FB1-3). With that, we can partially replicate the analysis. However, the data richness is not that good (the UMAPs are more *blooby*) so we preprocess the FASTQ files on our own, and used the processed adatas.

Once we have our own anndatas, the preprocessing is the following:
* Assign FB types based on Reynolds adatas to our adatas. Some cells will be unassigned.
* QC metrics 
* Filter genes (min counts = 30)
* Set raw X
* Normalize and log1p
* PCA + triku + neighbors + umap + leiden
* Use  `assign_cats` to assign leiden clusters to cell populations with selected markers (markers for each dataset may vary!)
* Filter adata to retain only fibroblasts
* Filter genes to remove 0 counts
* PCA + triku + neighbors + umap + leiden [leiden is not used here but may be used later]
* Check if strange populations appear and filter them in `assign_cats`, then repeat the last steps.

In [ ]:
os.getcwd()

In [ ]:
reynolds_dir = 'reynolds_2020'
os.makedirs(reynolds_dir, exist_ok=True)

### Making and saving the fb healthy dataset to zenodo

In [ ]:
# adata_reynolds = sc.read('submission_210120.h5ad', backup_url='https://zenodo.org/record/4536165/files/submission_210120.h5ad')
# adata_reynolds_fb = adata_reynolds[(adata_reynolds.obs['full_clustering'].isin(['F1', 'F2', 'F3'])) & 
#                                    (adata_reynolds.obs['Status'] == 'Healthy')]
# sc.pp.filter_genes(adata_reynolds_fb, min_counts=100)
# del adata_reynolds_fb.var

# for obs in ['mad_prd', 'Status', 'Site', 'Tissue', 'Enrichment', 'Location', 'Sex', 'Age', 'stage']:
#     del adata_reynolds_fb.obs[obs]
    
# adata_reynolds_fb.write_h5ad(reynolds_dir + '/reynolds_2020_fb_healthy.h5ad')

### Direct h5ad download

In [ ]:
adata_reynolds_fb_healthy = sc.read(reynolds_dir + '/reynolds_2020_fb_healthy.h5ad', 
                                    backup_url='https://zenodo.org/record/4605340/files/reynolds_2020_fb_healthy.h5ad?download=1')

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_healthy, min_counts=50)

In [ ]:
sc.pp.log1p(adata_reynolds_fb_healthy)
sc.pp.normalize_total(adata_reynolds_fb_healthy)

In [ ]:
df_batches = pd.DataFrame(np.unique(adata_reynolds_fb_healthy.obs['sample_id'], return_counts=True)).transpose()

In [ ]:
df_batches.sort_values(by=1, ascending=False)

In [ ]:
selected_samples = df_batches[df_batches[1] > 50][0].values

In [ ]:
adata_reynolds_fb_healthy = adata_reynolds_fb_healthy[adata_reynolds_fb_healthy.obs['sample_id'].isin(selected_samples)]  #selected_samples)]

In [ ]:
adata_reynolds_fb_healthy

In [ ]:
sc.pp.filter_genes(adata_reynolds_fb_healthy, min_counts=1)

In [ ]:
sc.pp.pca(adata_reynolds_fb_healthy, random_state=seed, n_comps=30)
sce.pp.bbknn(adata_reynolds_fb_healthy, metric='angular', batch_key='sample_id')
tk.tl.triku(adata_reynolds_fb_healthy, n_procs=1, random_state=seed, use_adata_knn=True)

In [ ]:
sc.tl.umap(adata_reynolds_fb_healthy, min_dist=0.1, random_state=seed)

In [ ]:
sc.tl.leiden(adata_reynolds_fb_healthy, resolution=1.5, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['leiden', 'sample_id', 'full_clustering'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['APCDD1', 'COL18A1', 'COMP', 'SLPI', 'WIF1'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['MT2A', 'CCL19', 'CCL2', 'CD46'], cmap=magma, use_raw=False)

In [ ]:
sc.pl.umap(adata_reynolds_fb_healthy, color=['POSTN', 'COMP', 'COCH'], cmap=magma, use_raw=False)

In [ ]:
adata_reynolds = sc.read('submission_210120.h5ad', backup_url='https://zenodo.org/record/4536165/files/submission_210120.h5ad')

In [ ]:
for adata_name in ['adata_reynolds']:
    sc.pp.filter_genes(eval(adata_name), min_counts=30)
    sc.pp.normalize_per_cell(eval(adata_name))
    sc.pp.log1p(eval(adata_name))
    sc.pp.highly_variable_genes(eval(adata_name))
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='sample_id', neighbors_within_batch=2)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
sc.pl.umap(adata_reynolds, color=['full_clustering'])

In [ ]:
sc.pl.umap(adata_reynolds, color=['BNIP3', 'BNIP3L'], cmap=magma, use_raw=False)

In [ ]:
adata_reynolds_fb = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['F1', 'F2', 'F3'])]
adata_reynolds_ve = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['VE1', 'VE2', 'VE3'])]
adata_reynolds_per = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['Pericyte_1', 'Pericyte_2'])]
adata_reynolds_krt = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['Differentiated_KC', 'Differentiated_KC*', 'Undifferentiated_KC'])]
adata_reynolds_lymphoid = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['Tc', 'Tc17_Th17', 'Tc_IL13_IL22', 'Th', 'Treg', 'ILC1_3', 'ILC1_NK', 'ILC2'])]
adata_reynolds_APC = adata_reynolds[adata_reynolds.obs['full_clustering'].isin(['Macro_1', 'Macro_2', 'Inf_mac', 'DC1', 'DC2', 'LC_1', 'LC_2', 'LC_3', 'Mono_mac', 'MigDC', 'MoDC_1', 'moDC_2', 'mo_DC3'])]

In [ ]:
samples, counts = np.unique(adata_reynolds_fb.obs['sample_id'], return_counts=True)
adata_reynolds_fb = adata_reynolds_fb[adata_reynolds_fb.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_ve.obs['sample_id'], return_counts=True)
adata_reynolds_ve = adata_reynolds_ve[adata_reynolds_ve.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_per.obs['sample_id'], return_counts=True)
adata_reynolds_per = adata_reynolds_per[adata_reynolds_per.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_krt.obs['sample_id'], return_counts=True)
adata_reynolds_krt = adata_reynolds_krt[adata_reynolds_krt.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_lymphoid.obs['sample_id'], return_counts=True)
adata_reynolds_lymphoid = adata_reynolds_lymphoid[adata_reynolds_lymphoid.obs['sample_id'].isin(samples[counts > 10])]

samples, counts = np.unique(adata_reynolds_APC.obs['sample_id'], return_counts=True)
adata_reynolds_APC = adata_reynolds_APC[adata_reynolds_APC.obs['sample_id'].isin(samples[counts > 10])]

In [ ]:
for adata_name in ['adata_reynolds_fb', 'adata_reynolds_ve', 'adata_reynolds_per', 'adata_reynolds_krt',
                   'adata_reynolds_lymphoid', 'adata_reynolds_APC']:
    sc.pp.filter_genes(eval(adata_name), min_counts=1)
    sc.pp.pca(eval(adata_name), random_state=seed, n_comps=30)
    sce.pp.bbknn(eval(adata_name), metric='angular', batch_key='sample_id', neighbors_within_batch=2)
    tk.tl.triku(eval(adata_name), n_procs=1, random_state=seed, use_adata_knn=True)
    sc.tl.umap(eval(adata_name), min_dist=0.05, random_state=seed)

In [ ]:
adata_reynolds_fb.write_h5ad(reynolds_dir + '/adata_reynolds_fb.h5ad')
adata_reynolds_ve.write_h5ad(reynolds_dir + '/adata_reynolds_ve.h5ad')
adata_reynolds_per.write_h5ad(reynolds_dir + '/adata_reynolds_per.h5ad')
adata_reynolds_krt.write_h5ad(reynolds_dir + '/adata_reynolds_krt.h5ad')
adata_reynolds_lymphoid.write_h5ad(reynolds_dir + '/adata_reynolds_lymphoid.h5ad')
adata_reynolds_APC.write_h5ad(reynolds_dir + '/adata_reynolds_APC.h5ad')

In [ ]:
adata_reynolds_fb = sc.read(reynolds_dir + '/adata_reynolds_fb.h5ad')
adata_reynolds_ve = sc.read(reynolds_dir + '/adata_reynolds_ve.h5ad')
adata_reynolds_per = sc.read(reynolds_dir + '/adata_reynolds_per.h5ad')
adata_reynolds_krt = sc.read(reynolds_dir + '/adata_reynolds_krt.h5ad')
adata_reynolds_lymphoid = sc.read(reynolds_dir + '/adata_reynolds_lymphoid.h5ad')
adata_reynolds_APC = sc.read(reynolds_dir + '/adata_reynolds_APC.h5ad')

In [ ]:
sc.pl.umap(adata_reynolds_fb, color=['full_clustering',  'sample_id'])

In [ ]:
dict_cats = {'Stress': ['ATF3', 'BTG2', 'CEBPB', 'CEBPD', 'CLDN4', 'CSRNP1', 'CTGF',
       'CXCL1', 'CXCL2', 'CYR61', 'DNAJA1', 'DNAJB1', 'DUSP1', 'DUSP2',
       'DUSP5', 'EGR1', 'ELF3', 'FOS', 'FOSB', 'GADD45B', 'GADD45G',
       'HSP90AA1', 'HSPA1A', 'HSPA1B', 'HSPB1', 'IER2', 'IER3', 'IFRD1',
       'IRF1', 'JUN', 'JUNB', 'JUND', 'KLF2', 'KLF4', 'KLF6', 'MAFF',
       'NFKBIA', 'NFKBIZ', 'NR4A1', 'NR4A2', 'PHLDA2', 'PIM1', 'PLAUR',
       'PLK3', 'PPP1R15A', 'RASD1', 'RHOB', 'SOCS3', 'TNFAIP3', 'UBC',
       'ZFP36'], 
        'Hypoxia': ['AC010655.2', 'ADIRF', 'ADM', 'AK2', 'AK4', 'ALDOC', 'ANGPTL4',
       'ANKRD37', 'ARID5A', 'ATF3', 'AXL', 'BAIAP2', 'BEND5', 'BIRC3',
       'BNIP3', 'BNIP3L', 'BNIP3P1', 'BX322234.1', 'CARHSP1', 'CAV1',
       'CAVIN1', 'CD44', 'CDON', 'CIRBP', 'CLEC2B', 'CNOT8', 'COL15A1',
       'COL27A1', 'CP', 'CRABP2', 'CRISPLD2', 'CRYBB2P1', 'DDIT4',
       'DDX41', 'DHRS13', 'DPYSL2', 'EEF1AKMT3', 'EGLN3', 'EHD2', 'ENO1',
       'ENO2', 'EPB41L4A-AS1', 'ERO1A', 'FAM162A', 'FAM210A', 'FGFR1',
       'FNBP1', 'GAPDH', 'GAS5', 'GBE1', 'GPRC5A', 'GYS1', 'HILPDA',
       'HK1', 'HLA-DRB1', 'HLA-J', 'HSD3B7', 'IER3', 'IGFBP3', 'IGFBP5',
       'IL6', 'INSIG2', 'IRF1', 'JAM2', 'JUN', 'KCTD11', 'KLF6', 'LDHA',
       'LGALS1', 'LIF', 'LMAN1', 'LOX', 'LOXL2', 'LOXL3', 'MEF2A', 'MXI1',
       'NBPF9', 'NDRG1', 'NGLY1', 'NRN1', 'OPTN', 'OSBPL5', 'P4HA1',
       'P4HA2', 'PDK1', 'PDK3', 'PDLIM2', 'PFKFB4', 'PGF', 'PGK1', 'PGM1',
       'PLIN2', 'PLOD2', 'PMP22', 'PPDPF', 'PPP1R15A', 'PPP1R18',
       'PPP1R3B', 'PTGIS', 'PTGS2', 'PTTG1IP', 'PYGL', 'RAB20', 'RASSF5',
       'RBM3', 'RBPJ', 'RORA', 'S100A10', 'SCD', 'SEC61G', 'SEPTIN9',
       'SFXN3', 'SH3BP5', 'SLC16A3', 'SLC2A1', 'SLC2A14', 'SLC2A3',
       'SLC38A5', 'SLC39A14', 'SLC6A6', 'SMARCA2', 'SNHG1', 'SNHG12',
       'SNHG5', 'SNHG7', 'SNHG8', 'SNX33', 'SPTBN1', 'SYNPO', 'TAF1D',
       'THBS2', 'TLCD3A', 'TMEM159', 'TMSB10', 'TNFAIP2', 'TNFAIP8',
       'TNIP1', 'TNS1', 'TPI1', 'UGP2', 'VEGFA', 'VIM', 'VKORC1',
       'WDR45B', 'WSB1', 'ZFAS1', 'ZFP36', 'ZNF292', 'ZNF395'], 
        'Normal': ['ACTB', 'ADRM1', 'AHSA1', 'AKR1C1', 'ANAPC11', 'AP2S1', 'APRT',
       'ARF4', 'ATP5F1B', 'ATP5MC1', 'ATP5MC3', 'ATP5MF', 'ATP5MG',
       'ATP5PD', 'ATP5PF', 'ATP6V0B', 'ATP6V1G1', 'AUP1', 'AURKAIP1',
       'BANF1', 'C1QBP', 'CALM2', 'CCT2', 'CCT3', 'CCT5', 'CCT6A', 'CCT7',
       'CEMIP', 'CFL1', 'CHCHD2', 'COX5A', 'COX6A1', 'COX6B1', 'CYC1',
       'DDX5', 'DNAJA1', 'EEF1B2', 'EIF3I', 'EIF4A1', 'ELOB', 'ELOC',
       'EMC7', 'ERH', 'FKBP1A', 'GNG5', 'GSTP1', 'GTF2A2', 'H3-3B',
       'HDAC2', 'HIF1A', 'HINT1', 'HMGB1', 'HNRNPM', 'HSBP1', 'HSP90AA1',
       'HSPA8', 'HSPA9', 'HSPD1', 'HSPE1', 'ID3', 'IFITM3', 'LAMTOR1',
       'LDHB', 'LSM7', 'MAGOH', 'MRPL20', 'MRPL3', 'MRPL36', 'MRPL41',
       'NAA10', 'NAMPT', 'NASP', 'NCL', 'NDUFA11', 'NDUFA13', 'NDUFA4',
       'NDUFA6', 'NDUFAB1', 'NDUFAF3', 'NDUFB2', 'NDUFB9', 'NDUFS5',
       'NDUFS6', 'NDUFS8', 'NHP2', 'NNMT', 'NOP10', 'OAZ1', 'PARK7',
       'PDCD5', 'PFDN2', 'PFN1', 'PHB', 'POLR2F', 'POLR2K', 'POLR2L',
       'PPIA', 'PPP4C', 'PRDX1', 'PRELID1', 'PRMT1', 'PSMA3', 'PSMA4',
       'PSMA5', 'PSMA7', 'PSMB3', 'PSMB5', 'PSMB6', 'PSMC5', 'PSMD2',
       'PSMD7', 'PSMD8', 'PUF60', 'RAB5C', 'RAN', 'RANBP1', 'RTRAF',
       'SEC61B', 'SERPINE2', 'SF3B6', 'SKP1', 'SLC25A5', 'SLC3A2',
       'SNRPG', 'SNU13', 'SOD2', 'SPCS1', 'SRP14', 'SSB', 'SSBP1',
       'TALDO1', 'TMA7', 'TMBIM4', 'TMEM147', 'TMEM70', 'TPM4', 'TRMT112',
       'TYMP', 'UBL5', 'UQCR11', 'XRCC5', 'YBX1', 'YWHAB', 'ZFAND6']
            }
assign_cats(adata_reynolds_fb, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')

In [ ]:
sc.pl.umap(adata_reynolds_fb, color=['full_clustering', 'Status', 'hypoxia_stress'], cmap=magma, use_raw=False)

In [ ]:
assign_cats(adata_reynolds_ve, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_ve, color=['full_clustering', 'Status', 'hypoxia_stress'], cmap=magma, use_raw=False)

In [ ]:
assign_cats(adata_reynolds_per, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_per, color=['full_clustering', 'Status', 'hypoxia_stress'], cmap=magma, use_raw=False)

In [ ]:
assign_cats(adata_reynolds_krt, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_krt, color=['full_clustering', 'Status', 'hypoxia_stress'], cmap=magma, use_raw=False)

In [ ]:
assign_cats(adata_reynolds_lymphoid, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_lymphoid, color=['full_clustering', 'Status', 'hypoxia_stress'], cmap=magma, use_raw=False)

In [ ]:
assign_cats(adata_reynolds_APC, dict_cats=dict_cats, column_groupby='leiden', intermediate_states=False, min_score=0.55,
            key_added='hypoxia_stress')
sc.pl.umap(adata_reynolds_APC, color=['full_clustering', 'Status', 'hypoxia_stress'], cmap=magma, use_raw=False)